In [4]:
import pandas as pd
import requests
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import pytz

In [5]:
API_KEY = "2f55d162e4a594dc6c1cb30c7c36c6ea"
BASE_URL = "https://api.openweathermap.org/data/2.5/"

In [ ]:
def get_current_weather(city):
    url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    data = response.json()
    return {
        'city': data['name'],
        'current_temp' : round(data['main']['temp']),
        'feels_like' : round(data['main']['feels_like']),
        'temp_min' : data['name'],
        'current_temp' : round(data['main']['temp']),
        'feels_like' : round(data['main']['feels_like']),
        'temp_min' : round(data['main']['temp_min']),
        'temp_max' : round(data['main']['temp_min']),
        'humidity' : round(data['main']['humidity']),
        'description' : data['weather'][0]['description'],
        'country' : data["sys"]['country'],
        "wind_gust_dir" : data["wind"]["deg"],
        "pressure" : data["main"]["pressure"],
        "Wind_Gust_Speed" : data["wind"]["speed"]
    }

In [7]:
def read_historical_data(filename):
    df = pd.read_csv(filename)
    df = df.dropna()
    df = df.drop_duplicates()
    return df

In [8]:
def prepare_data(data):
    le = LabelEncoder()
    data["WindGustDir"] = le.fit_transform(data["WindGustDir"])
    data["RainTomorrow"] = le.fit_transform(data["RainTomorrow"])
    
    X = data[["MinTemp", "MaxTemp","WindGustDir","WindGustSpeed","Humidity","Pressure","Temp"]]
    y = data["RainTomorrow"]

    return X, y, le

In [9]:
def train_rain_model(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    print("Mean Squared Error for Rain Model")

    print(mean_squared_error(y_test, y_pred))

    return model

In [ ]:
def prepare_regression_data(data, feature):
    X, y = [], []

    for i in range(len(data) - 1):
        X.append(data[feature].iloc[i])
        y.append(data[feature].iloc[i+1])

    X = np.array(X).reshape(-1, 1)
    y = np.array(y)
    return X, y

In [11]:
def train_regression_model(X, y):
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X,y)
    return model

In [12]:
def predict_future(model, current_value):
    predictions = [current_value]

    for i in range(5):
        next_value = model.predict(np.array([[predictions[-1]]]))
        predictions.append(next_value[0])
    
    return predictions[1:]

In [13]:
def weather_view():
    city = input("Enter any City Name : ")
    current_weather = get_current_weather(city)

    historical_data = read_historical_data("/home/daniar/Vs_Code/Kuliah/Coba _coba/Forecasting_Weather/data/weather.csv")

    X, y, le = prepare_data(historical_data)

    rain_model = train_rain_model(X, y)

    wind_deg = current_weather['wind_gust_dir'] % 360
    compass_points = [
    ("N", 0, 11.25),
    ("NNE", 11.25, 33.75),
    ("NE", 33.75, 56.25),
    ("ENE", 56.25, 78.75),
    ("E", 78.75, 101.25),
    ("ESE", 101.25, 123.75),
    ("SE", 123.75, 146.25),
    ("SSE", 146.25, 168.75),
    ("S", 168.75, 191.25),
    ("SSW", 191.25, 213.75),
    ("SW", 213.75, 236.25),
    ("WSW", 236.25, 258.75),
    ("W", 258.75, 281.25),
    ("WNW", 281.25, 303.75),
    ("NW", 303.75, 326.25),
    ("NNW", 326.25, 348.75)
    ]

    compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)

    compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1
    current_data = {
        "MinTemp" : current_weather['temp_min'],
        "MaxTemp" : current_weather['temp_max'],
        "WindGustDir" : compass_direction_encoded,
        "WindGustSpeed" : current_weather["Wind_Gust_Speed"],
        "Humidity" : current_weather["humidity"],
        "Pressure" : current_weather["pressure"],
        "Temp" : current_weather["current_temp"]
    }

    current_df = pd.DataFrame([current_data])

    rain_prediction = rain_model.predict(current_df)[0]

    X_temp, y_temp = prepare_regression_data(historical_data, "Temp")

    X_hum, y_hum = prepare_regression_data(historical_data, "Humidity")

    temp_model = train_regression_model(X_temp, y_temp)

    hum_model = train_regression_model(X_hum, y_hum)

    future_temp = predict_future(temp_model, current_weather['temp_min'])
    future_humidity = predict_future(hum_model, current_weather['humidity'])

    timezone = pytz.timezone("Asia/Jakarta")
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)

    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

    print(f"City: {city}, {current_weather["country"]}")
    print(f"Current Temperature : {current_weather["current_temp"]}")
    print(f"Feels Like: {current_weather["feels_like"]}")
    print(f"Minimum Temperatur: {current_weather["temp_min"]}°C")
    print(f"Maximum Temperatur: {current_weather["temp_max"]}°C")
    print(f"Humadity : {current_weather["humidity"]}%")
    print(f"Weather Prediction : {current_weather["description"]}")
    print(f"Rain Prediction : {'Yes' if rain_prediction else 'No'}")

    print("\nFuture Temperature Prediction")
    
    for  time, temp in zip(future_times, future_temp):
        print(f"{time}: {round(temp, 1)}°C")
    
    print("\nFuture Humadity Prediction:")

    for time, humidity in zip(future_times, future_humidity):
        print(f"{time}: {round(humidity, 1)}%")

weather_view()

Mean Squared Error for Rain Model
0.1506849315068493
City: Pasuruan, ID
Current Temperature : 34
Feels Like: 41
Minimum Temperatur: 34°C
Maximum Temperatur: 34°C
Humadity : 83%
Weather Prediction : light rain
Rain Prediction : Yes

Future Temperature Prediction
11:00: 31.2°C
12:00: 30.4°C
13:00: 30.5°C
14:00: 29.5°C
15:00: 26.5°C

Future Humadity Prediction:
11:00: 54.1%
12:00: 58.1%
13:00: 54.2%
14:00: 58.1%
15:00: 54.2%


In [ ]:
def run_weather_prediction(city_name: str, historical_csv: str = "data/weather.csv"):
    API_KEY = "2f55d162e4a594dc6c1cb30c7c36c6ea"
    BASE_URL = "https://api.openweathermap.org/data/2.5/"
    
    def get_current_weather(city):
        url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
        response = requests.get(url)
        data = response.json()
        return {
            'city': data['name'],
            'current_temp' : round(data['main']['temp']),
            'feels_like' : round(data['main']['feels_like']),
            'temp_min' : data['name'],
            'current_temp' : round(data['main']['temp']),
            'feels_like' : round(data['main']['feels_like']),
            'temp_min' : round(data['main']['temp_min']),
            'temp_max' : round(data['main']['temp_min']),
            'humidity' : round(data['main']['humidity']),
            'description' : data['weather'][0]['description'],
            'country' : data["sys"]['country'],
            "wind_gust_dir" : data["wind"]["deg"],
            "pressure" : data["main"]["pressure"],
            "Wind_Gust_Speed" : data["wind"]["speed"]
        }
    
    def read_historical_data(filename):
        df = pd.read_csv(filename)
        df = df.dropna()
        df = df.drop_duplicates()
        return df
    
    def prepare_data(data):
        le = LabelEncoder()
        data["WindGustDir"] = le.fit_transform(data["WindGustDir"])
        data["RainTomorrow"] = le.fit_transform(data["RainTomorrow"])
        
        X = data[["MinTemp", "MaxTemp","WindGustDir","WindGustSpeed","Humidity","Pressure","Temp"]]
        y = data["RainTomorrow"]

        return X, y, le

    def train_rain_model(X,y):
        X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        print("Mean Squared Error for Rain Model")

        print(mean_squared_error(y_test, y_pred))

        return model

    def prepare_regression_data(data, feature):
        X, y = [], []

        for i in range(len(data) - 1):
            X.append(data[feature].iloc[i])
            y.append(data[feature].iloc[i+1])

        X = np.array(X).reshape(-1, 1)
        y = np.array(y)
        return X, y
    
    def train_regression_model(X, y):
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X,y)
        return model
    
    def predict_future(model, current_value):
        predictions = [current_value]

        for i in range(5):
            next_value = model.predict(np.array([[predictions[-1]]]))
            predictions.append(next_value[0])
        
        return predictions[1:]
    
    def weather_view():
        city = input("Enter any City Name : ")
        current_weather = get_current_weather(city)

        historical_data = read_historical_data("/home/daniar/Vs_Code/Kuliah/Coba _coba/Forecasting_Weather/data/weather.csv")

        X, y, le = prepare_data(historical_data)

        rain_model = train_rain_model(X, y)

        wind_deg = current_weather['wind_gust_dir'] % 360
        compass_points = [
        ("N", 0, 11.25),
        ("NNE", 11.25, 33.75),
        ("NE", 33.75, 56.25),
        ("ENE", 56.25, 78.75),
        ("E", 78.75, 101.25),
        ("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25),
        ("SSE", 146.25, 168.75),
        ("S", 168.75, 191.25),
        ("SSW", 191.25, 213.75),
        ("SW", 213.75, 236.25),
        ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25),
        ("WNW", 281.25, 303.75),
        ("NW", 303.75, 326.25),
        ("NNW", 326.25, 348.75)
        ]

        compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)

        compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1
        current_data = {
            "MinTemp" : current_weather['temp_min'],
            "MaxTemp" : current_weather['temp_max'],
            "WindGustDir" : compass_direction_encoded,
            "WindGustSpeed" : current_weather["Wind_Gust_Speed"],
            "Humidity" : current_weather["humidity"],
            "Pressure" : current_weather["pressure"],
            "Temp" : current_weather["current_temp"]
        }

        current_df = pd.DataFrame([current_data])

        rain_prediction = rain_model.predict(current_df)[0]

        X_temp, y_temp = prepare_regression_data(historical_data, "Temp")

        X_hum, y_hum = prepare_regression_data(historical_data, "Humidity")

        temp_model = train_regression_model(X_temp, y_temp)

        hum_model = train_regression_model(X_hum, y_hum)

        future_temp = predict_future(temp_model, current_weather['temp_min'])
        future_humidity = predict_future(hum_model, current_weather['humidity'])

        timezone = pytz.timezone("Asia/Jakarta")
        now = datetime.now(timezone)
        next_hour = now + timedelta(hours=1)
        next_hour = next_hour.replace(minute=0, second=0, microsecond=0)

        future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

        print(f"City: {city}, {current_weather["country"]}")
        print(f"Current Temperature : {current_weather["current_temp"]}")
        print(f"Feels Like: {current_weather["feels_like"]}")
        print(f"Minimum Temperatur: {current_weather["temp_min"]}°C")
        print(f"Maximum Temperatur: {current_weather["temp_max"]}°C")
        print(f"Humadity : {current_weather["humidity"]}%")
        print(f"Weather Prediction : {current_weather["description"]}")
        print(f"Rain Prediction : {'Yes' if rain_prediction else 'No'}")

        print("\nFuture Temperature Prediction")
        
        for  time, temp in zip(future_times, future_temp):
            print(f"{time}: {round(temp, 1)}°C")
        
        print("\nFuture Humadity Prediction:")

        for time, humidity in zip(future_times, future_humidity):
            print(f"{time}: {round(humidity, 1)}%")

    weather_view()

: 

In [ ]:
def read_historical_data(filename):
    df = pd.read_csv(filename)
    df = df.dropna()
    df = df.drop_duplicates()
    return df

def train_rain_model(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    print("Mean Squared Error for Rain Model")

    print(mean_squared_error(y_test, y_pred))

    return model

def prepare_regression_data(data, feature):
    X, y = [], []

    for i in range(len(data) - 1):
        X.append(data[feature].iloc[i])
        y.append(data[feature].iloc[i+1])

    X = np.array(X).reshape(-1, 1)
    y = np.array(y)
    return X, y

def train_regression_model(X, y):
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X,y)
    return model

def weather_view():
    city = input("Enter any City Name : ")
    current_weather = get_current_weather(city)

    historical_data = read_historical_data("/home/daniar/Vs_Code/Kuliah/Coba _coba/Forecasting_Weather/data/weather.csv")

    X, y, le = prepare_data(historical_data)

    rain_model = train_rain_model(X, y)

    wind_deg = current_weather['wind_gust_dir'] % 360
    compass_points = [
    ("N", 0, 11.25),
    ("NNE", 11.25, 33.75),
    ("NE", 33.75, 56.25),
    ("ENE", 56.25, 78.75),
    ("E", 78.75, 101.25),
    ("ESE", 101.25, 123.75),
    ("SE", 123.75, 146.25),
    ("SSE", 146.25, 168.75),
    ("S", 168.75, 191.25),
    ("SSW", 191.25, 213.75),
    ("SW", 213.75, 236.25),
    ("WSW", 236.25, 258.75),
    ("W", 258.75, 281.25),
    ("WNW", 281.25, 303.75),
    ("NW", 303.75, 326.25),
    ("NNW", 326.25, 348.75)
    ]

    compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)

    compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1
    current_data = {
        "MinTemp" : current_weather['temp_min'],
        "MaxTemp" : current_weather['temp_max'],
        "WindGustDir" : compass_direction_encoded,
        "WindGustSpeed" : current_weather["Wind_Gust_Speed"],
        "Humidity" : current_weather["humidity"],
        "Pressure" : current_weather["pressure"],
        "Temp" : current_weather["current_temp"]
    }

    current_df = pd.DataFrame([current_data])

    rain_prediction = rain_model.predict(current_df)[0]

    X_temp, y_temp = prepare_regression_data(historical_data, "Temp")

    X_hum, y_hum = prepare_regression_data(historical_data, "Humidity")

    temp_model = train_regression_model(X_temp, y_temp)

    hum_model = train_regression_model(X_hum, y_hum)

    future_temp = predict_future(temp_model, current_weather['temp_min'])
    future_humidity = predict_future(hum_model, current_weather['humidity'])

    timezone = pytz.timezone("Asia/Jakarta")
    now = datetime.now(timezone)
    next_hour = now + timedelta(hours=1)
    next_hour = next_hour.replace(minute=0, second=0, microsecond=0)

    future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

    print(f"City: {city}, {current_weather["country"]}")
    print(f"Current Temperature : {current_weather["current_temp"]}")
    print(f"Feels Like: {current_weather["feels_like"]}")
    print(f"Minimum Temperatur: {current_weather["temp_min"]}°C")
    print(f"Maximum Temperatur: {current_weather["temp_max"]}°C")
    print(f"Humadity : {current_weather["humidity"]}%")
    print(f"Weather Prediction : {current_weather["description"]}")
    print(f"Rain Prediction : {'Yes' if rain_prediction else 'No'}")

    print("\nFuture Temperature Prediction")
    
    for  time, temp in zip(future_times, future_temp):
        print(f"{time}: {round(temp, 1)}°C")
    
    print("\nFuture Humadity Prediction:")

    for time, humidity in zip(future_times, future_humidity):
        print(f"{time}: {round(humidity, 1)}%")

weather_view()

Timezone lokal Anda adalah: Asia/Jakarta


In [1]:
import numpy as np
import pandas as pd

# Contoh arah mata angin dalam data
df = pd.DataFrame({
    "WindGustDir": ["N", "NNE", "NE", "E", "SE", "S", "SW", "W", "NW", "NNW"]
})

# Mapping arah mata angin ke derajat
compass_to_degrees = {
    "N": 0, "NNE": 22.5, "NE": 45, "ENE": 67.5,
    "E": 90, "ESE": 112.5, "SE": 135, "SSE": 157.5,
    "S": 180, "SSW": 202.5, "SW": 225, "WSW": 247.5,
    "W": 270, "WNW": 292.5, "NW": 315, "NNW": 337.5
}

# Ubah ke derajat
df['WindDirDeg'] = df['WindGustDir'].map(compass_to_degrees)

# Encode ke sin dan cos
df['WindDir_sin'] = np.sin(np.deg2rad(df['WindDirDeg']))
df['WindDir_cos'] = np.cos(np.deg2rad(df['WindDirDeg']))

print(df)


  WindGustDir  WindDirDeg   WindDir_sin   WindDir_cos
0           N         0.0  0.000000e+00  1.000000e+00
1         NNE        22.5  3.826834e-01  9.238795e-01
2          NE        45.0  7.071068e-01  7.071068e-01
3           E        90.0  1.000000e+00  6.123234e-17
4          SE       135.0  7.071068e-01 -7.071068e-01
5           S       180.0  1.224647e-16 -1.000000e+00
6          SW       225.0 -7.071068e-01 -7.071068e-01
7           W       270.0 -1.000000e+00 -1.836970e-16
8          NW       315.0 -7.071068e-01  7.071068e-01
9         NNW       337.5 -3.826834e-01  9.238795e-01
